In [31]:
import pprint

import numpy as np
import pandas as pd
import xarray as xr
import lightgbm as lgb

from otb.tasks import TaskApi
from otb.benchmark.models import (
    MeanRegressionModel,
    MeanWindowForecastingModel,
    AWTModel,
    MacroMeterologicalModel,
    OffshoreMacroMeterologicalModel
)

In [2]:
from otb.benchmark.bench_runner import run_benchmarks

In [3]:
results = run_benchmarks(verbose=False, write_metrics=True, metrics_fp="/home/cjellen/sources/github/ot-bench/otb/benchmark/benchmark_metrics.json")

In [4]:
%matplotlib inline

In [5]:
pprinter = pprint.PrettyPrinter(indent=4, width=120, compact=True)

### load the tasks

In [6]:
task_api = TaskApi()

List all supported tasks

In [7]:
task_api.list_tasks()

['regression.usna_cn2_sm.full.Cn2_3m',
 'regression.usna_cn2_lg.full.Cn2_3m',
 'forecasting.mlo_cn2.dropna.Cn2_15m',
 'regression.mlo_cn2.full.Cn2_15m',
 'regression.mlo_cn2.dropna.Cn2_15m',
 'forecasting.usna_cn2_sm.full.Cn2_3m']

Get details for the `mlo_cn2` regression task, with missing values dropped

In [8]:
task = task_api.get_task("regression.mlo_cn2.dropna.Cn2_15m")

In [9]:
type(task)

otb.tasks.tasks.RegressionTask

In [10]:
task_info = task.get_all_info()
pprinter.pprint(task_info)

{   'description': 'Regression task for MLO Cn2 data, where the last 12 days are set aside for evaluation',
    'description_long': 'This dataset evaluates regression approaches for predicting the extent of optical turbulence, '
                        'as measured by Cn2 at an elevation of 15m. Optical turbulence on data collected at the Mauna '
                        'Loa Solar Observatory between 27 July 2006 and 8 August 2006, inclusive, are used to evaluate '
                        'prediction accuracy under the root-mean square error metric.',
    'dropna': True,
    'ds_name': 'mlo_cn2',
    'eval_metrics': ['root_mean_square_error', 'r2_score', 'mean_absolute_error', 'mean_absolute_percentage_error'],
    'log_transform': True,
    'obs_lat': 19.53,
    'obs_lon': -155.57,
    'obs_tz': 'US/Hawaii',
    'remove': ['base_time', 'Cn2_6m', 'Cn2_15m', 'Cn2_25m'],
    'target': 'Cn2_15m',
    'test_idx': ['8367:10367'],
    'train_idx': ['0:8367'],
    'val_idx': ['10367:13943']}


Get the training data

In [11]:
X_train, y_train = task.get_train_data(data_type="pd")

Train your model

In [12]:
model = lgb.LGBMRegressor()

In [13]:
model.fit(X_train, y_train)

LGBMRegressor()

Evaluate your model

In [14]:
task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': {'metric_value': 0.21550584034473647,
  'valid_predictions': 2449},
 'r2_score': {'metric_value': 0.8936031659084048, 'valid_predictions': 2449},
 'mean_absolute_error': {'metric_value': 0.15718607854371786,
  'valid_predictions': 2449},
 'mean_absolute_percentage_error': {'metric_value': 0.011320035736497307,
  'valid_predictions': 2449}}

Compare against benchmarks

In [15]:
height_of_observation = float(task_info["target"].split("Cn2_")[-1].split("m")[0])
obs_tz = task_info["obs_tz"]
obs_lat = task_info["obs_lat"]
obs_lon = task_info["obs_lon"]

In [22]:
mm_model = MacroMeterologicalModel(
    name="macro_met_model",
    timezone=obs_tz,
    obs_lat=obs_lat,
    obs_lon=obs_lon,
    air_temperature_col_name="T_2m",
    humidity_col_name="RH_2m",
    wind_speed_col_name="Spd_10m",
    time_col_name="time",
    height_of_observation=15,
    enforce_dynamic_range=True,
    constant_adjustment=True,
    use_log10=True,
)

In [23]:
task.evaluate_model(predict_call=mm_model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': {'metric_value': 0.7138353117427121,
  'valid_predictions': 813},
 'r2_score': {'metric_value': -1.0279853591950694, 'valid_predictions': 813},
 'mean_absolute_error': {'metric_value': 0.5874813616291615,
  'valid_predictions': 813},
 'mean_absolute_percentage_error': {'metric_value': 0.04435200340221298,
  'valid_predictions': 813}}

In [24]:
omm_model = OffshoreMacroMeterologicalModel(
    name="offshore_macro_met_model",
    timezone="US/Hawaii",
    obs_lat=19.53,
    obs_lon=-155.57,
    air_temperature_col_name="T_2m",
    humidity_col_name="RH_2m",
    wind_speed_col_name="Spd_10m",
    time_col_name="time",
    height_of_observation=15,
    enforce_dynamic_range=True,
    constant_adjustment=True,
    use_log10=True,
)

In [25]:
task.evaluate_model(predict_call=omm_model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': {'metric_value': 0.8157009064248446,
  'valid_predictions': 2274},
 'r2_score': {'metric_value': -0.4749983894964831, 'valid_predictions': 2274},
 'mean_absolute_error': {'metric_value': 0.6221959463893278,
  'valid_predictions': 2274},
 'mean_absolute_percentage_error': {'metric_value': 0.04375820062480316,
  'valid_predictions': 2274}}

Get details for the full `usna_cn2_sm` regression task

In [26]:
task = task_api.get_task("regression.usna_cn2_sm.full.Cn2_3m")

In [27]:
type(task)

otb.tasks.tasks.RegressionTask

In [28]:
task.get_all_info()

{'description': 'Regression task for USNA Cn2 small data, where the last 15 days are set aside for evaluation.',
 'description_long': "This dataset evaluates regression approaches for predicting the extent of optical turbulence, as measured by Cn2 at an elevation of 3m. Optical turbulence on data collected at the United States Naval Academy, across the Severn River between Santee Basin and the Waterfront Readiness Center. This dataset contains measurements between 01 June 2021 and 31 August 2021, inclusive. Meterological and Oceanographic measurements are collated using the scintillometer's internal clock at a 6-minute frequency. Prediction accuracy is evaluated  under the root-mean square error metric.",
 'ds_name': 'usna_cn2_sm',
 'obs_tz': 'US/Eastern',
 'obs_lat': 38.98,
 'obs_lon': -76.48,
 'train_idx': ['0:14640'],
 'test_idx': ['14640:18000'],
 'val_idx': ['18000:22081'],
 'dropna': False,
 'log_transform': True,
 'eval_metrics': ['root_mean_square_error',
  'r2_score',
  'mean_

Get the training data

In [29]:
X_train, y_train = task.get_train_data(data_type="pd")

Define or train your model

In [32]:
model = AWTModel(
    name="awt_model",
    air_temperature_col_name="T_5m",
    water_temperature_col_name="T_0m"
)

task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

In [37]:
model = MeanRegressionModel(
    name="mean_model",
)

model.train(X_train, y_train)

task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': {'metric_value': 0.4860083120440881,
  'valid_predictions': 4080},
 'r2_score': {'metric_value': -0.10302263599092476, 'valid_predictions': 4080},
 'mean_absolute_error': {'metric_value': 0.3903624308911522,
  'valid_predictions': 4080},
 'mean_absolute_percentage_error': {'metric_value': 0.027622987088901995,
  'valid_predictions': 4080}}

In [ ]:
model = MeanRegressionModel(
    name="mean_model",
)

model.train(X_train, y_train)

task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

Get details for the `mlo_cn2` forecasting task, with missing values dropped

In [38]:
task = task_api.get_task("forecasting.mlo_cn2.dropna.Cn2_15m")

In [39]:
type(task)

otb.tasks.tasks.ForecastingTask

In [40]:
task.get_all_info()

{'description': 'Forecasting task for MLO Cn2 data, where the last 12 days are set aside for evaluation',
 'description_long': 'This dataset evaluates forecasting approaches for predicting the extent of optical turbulence, as measured by Cn2 at an elevation of 15m. Optical turbulence on data collected at the Mauna Loa Solar Observatory between 27 July 2006 and 8 August 2006, inclusive, are used to evaluate prediction accuracy for measured Cn2 one observation (6 minutes) in the future. The forecasting task makes the last 4 observations of meterological parameters and measured Cn2 available as inputs to forecasting models.',
 'ds_name': 'mlo_cn2',
 'obs_tz': 'US/Hawaii',
 'obs_lat': 19.53,
 'obs_lon': -155.57,
 'train_idx': ['0:8367'],
 'test_idx': ['8367:10367'],
 'val_idx': ['10367:13943'],
 'dropna': True,
 'log_transform': True,
 'eval_metrics': ['root_mean_square_error',
  'r2_score',
  'mean_absolute_error',
  'mean_absolute_percentage_error'],
 'window_size': 4,
 'forecast_horizon

Get the training data

In [41]:
X_train, y_train = task.get_train_data(data_type="pd")

In [42]:
X_train, y_train = task.prepare_forecasting_data(X_train, y_train)

Train your model

In [44]:
model = lgb.LGBMRegressor(random_state=2020)

In [45]:
model.fit(X_train, y_train)

LGBMRegressor(random_state=2020)

Evaluate your model

In [64]:
task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': (0.66010934, 2444),
 'r2_score': (-1.5093609997407498e-06, 2444),
 'mean_absolute_error': (0.5284559, 2444),
 'mean_absolute_percentage_error': (0.03816938, 2444)}

In [46]:
model = MeanWindowForecastingModel(
    name="mean_window_model",
    target_name=task.get_target_name(),
)

task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': {'metric_value': 0.2779041826725006,
  'valid_predictions': 2444},
 'r2_score': {'metric_value': 0.8227610719548378, 'valid_predictions': 2444},
 'mean_absolute_error': {'metric_value': 0.196153461933136,
  'valid_predictions': 2444},
 'mean_absolute_percentage_error': {'metric_value': 0.014085889793932438,
  'valid_predictions': 2444}}

Compare against benchmarks

Get details for the `usna_cn2_sm` forecasting task

In [47]:
task = task_api.get_task("forecasting.usna_cn2_sm.full.Cn2_3m")

In [48]:
type(task)

otb.tasks.tasks.ForecastingTask

In [49]:
task.get_all_info()

{'description': 'Forecasting task for USNA Cn2 small data, where the last 15 days are set aside for evaluation.',
 'description_long': "This dataset evaluates forecasting approaches for predicting the extent of optical turbulence, as measured by Cn2 at an elevation of 3m. Optical turbulence on data collected at the United States Naval Academy, across the Severn River between Santee Basin and the Waterfront Readiness Center. This dataset contains measurements between 01 June 2021 and 31 August 2021, inclusive. Meterological and Oceanographic measurements are collated using the scintillometer's internal clock at a 6-minute frequency. Prediction accuracy is evaluated  under the root-mean square error metric.",
 'ds_name': 'usna_cn2_sm',
 'obs_tz': 'US/Eastern',
 'obs_lat': 38.98,
 'obs_lon': -76.48,
 'train_idx': ['0:14640'],
 'test_idx': ['14640:18000'],
 'val_idx': ['18000:22081'],
 'dropna': False,
 'log_transform': True,
 'eval_metrics': ['root_mean_square_error',
  'r2_score',
  'mea

Get the training data

In [50]:
X_train, y_train = task.get_train_data(data_type="pd")

In [51]:
X_train, y_train = task.prepare_forecasting_data(X_train, y_train)

Train your model

In [52]:
model = lgb.LGBMRegressor(random_state=2020)

In [53]:
model.fit(X_train, y_train)

LGBMRegressor(random_state=2020)

Evaluate your model

In [54]:
task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': {'metric_value': 0.16888039580056738,
  'valid_predictions': 4073},
 'r2_score': {'metric_value': 0.8665714886554514, 'valid_predictions': 4073},
 'mean_absolute_error': {'metric_value': 0.08753583133425404,
  'valid_predictions': 4073},
 'mean_absolute_percentage_error': {'metric_value': 0.00621424978647618,
  'valid_predictions': 4073}}

Compare against benchmarks